### azure pre-requesite

In [ ]:
# get connection_string
import os
connection_string = os.getenv("CONN_STRING")

In [ ]:
# import container client and enumerate blob within container
from azure.storage.blob import ContainerClient

def blobInContainer(container,c_string):
    campaign0_container = ContainerClient.from_connection_string(conn_str=c_string, container_name=container)
    blob_list = campaign0_container.list_blobs()
    blob_names_list = []
    for blob in blob_list:
        blob_names_list.append(blob.name)
    return blob_names_list

blobs_campaign0 = blobInContainer('campaign0',connection_string)
print(blobs_campaign0)

# Get Blob Video info
from azure.storage.blob import BlobClient

def blobInfos(blob_name,c_string):
    blob_video = BlobClient.from_connection_string(conn_str=c_string,container_name="campaign0", blob_name=blob_video_name)
    blob_video_url = blob_video.url
    blob_video_prop = blob_video.get_blob_properties()
    blob_video_prop_keys = blob_video_prop.keys()
    print("blob name:",blob_video_name)
    print("blob URL:",blob_video_url)
    print("blob properties:", blob_video_prop)
    print("blob properties keys:", blob_video_prop_keys)

blob_video_name = 'vid1-4K4.mp4'
blobInfos(blob_video_name,connection_string)

In [ ]:
from azure.storage.blob import BlobClient
blob_video0 = BlobClient.from_connection_string(conn_str=connection_string,container_name="campaign0", blob_name="vid1-4K4.mp4")

''' function to download blob from Azure to local file system'''
''' parameter is a blob client object from azure storage sdk'''
def downloadBlob(blobclient):
    with open("/tmp/"+blobclient.blob_name, "wb") as my_blob_dl:
        blob_data = blobclient.download_blob()
        blob_data.readinto(my_blob_dl)
    print("Blob %s downloaded" %blobclient.blob_name)
    print("Blob path: /tmp/%s" %blobclient.blob_name)

downloadBlob(blob_video0)
# this function should handle connection issue

### REST requests

In [ ]:
#https://stackoverflow.com/questions/3777301/how-to-call-a-shell-script-from-python-code
#https://stackoverflow.com/questions/325463/launch-a-shell-command-with-in-a-python-script-wait-for-the-termination-and-ret

import subprocess

''' getPrediction function sends curl request to Surfrider AI'''
''' video name is the name of a locally downloaded video from Azure'''
''' video name is passed as an argument to curl_request script '''
''' curl_request script sends actual POST request to Surfrider AI'''
def getPrediction(video_name):
    curl_request_script = ['./curl_request_param.sh',video_name]
    output = []
    request_answer = subprocess.Popen(curl_request_script, stdout=subprocess.PIPE)
    for line in request_answer.stdout:
        print(line)
        output.append(line)
    return output

# this function should handle connection issue

In [ ]:
# get predictions from AI on vid1-4K4.MP4
prediction = getPrediction('vid1-4K2.mp4')
prediction

### JSON Decode

In [ ]:
import json

''' jsonPrediction cast a prediction from getPrediction function '''
''' and cast the list prediction as a string then dictionnary with json_loads '''
def jsonPrediction(pred):
    string_prediction = str(pred[0])[2:-3] #removing 2 x first and 3 last characters of pred
    json_prediction = json.loads(string_prediction)
    return json_prediction

In [ ]:
json_prediction = jsonPrediction(prediction)
json_prediction

In [ ]:
for frame_to_box in json_prediction['detected_trash']:
    print(frame_to_box)
    print(frame_to_box['label'])

In [ ]:
''' getTrashLabel return label from a frame_to_box'''
def getTrashLabel(frame_2_box):
    return frame_to_box['label']

test_f2b = {'frame_to_box': {'2': [0.45, 0.44, 0.5, 0.5]}, 'id': 0, 'label': 'fragments'}
getTrashLabel(test_f2b['frame_to_box'])

### requests - skip for now

In [ ]:
# Make first GET Request using requests module
import requests
r_get = requests.get('https://github.com/timeline.json')
print(r_get.text)
# Make first POST Request using requests module
r_post = requests.post("http://httpbin.org/post")
print(r_post.text) # response in text format
print(r_post.json()) # response is json formated

# Video POST request example
desktop = '/Users/clement/Desktop/'
video = 'vid1-4K.MP4'
video_path = os.path.join(desktop,video)
url = 'http://aiapisurfrider.northeurope.cloudapp.azure.com:5000'
myfile = {'file': open(video_path ,'rb')}
#myairequest = requests.post(url, files = myfile)

# Image POST request example
desktop = '/Users/clement/Desktop/'
img = 'G0061735.JPG'
img_path = os.path.join(desktop,img)
url = 'http://aiapisurfrider.northeurope.cloudapp.azure.com:5000'
myfile = {'file': open(img_path ,'rb')}
myairequest = requests.post(url, files = myfile)

### SQL Operations

In [ ]:
# import sql odbc driver and connect to SQL DB
import os
import pyodbc
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
username = os.getenv("USERNAME")
password = os.getenv("PWD")
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
# Sample Query on dbo.Trash_Type
cursor.execute("SELECT * FROM dbo.Trash_Type")
row = cursor.fetchone()
while row:
    print (str(row[0]))
    print (str(row[1]))
    row = cursor.fetchone()

In [ ]:
def mapLabel2TrashId(label):
    switcher = { 
    "Fishing or Hunting":"89B44BAA-69AA-4109-891A-128E012E7E07",
    "Food Packaging":"185FEFA2-EEF2-47A8-873E-26032A4BB3C3",
    "Unknown":"BB4DEA69-218A-40CC-A000-2AE17C37152C",
    "Industrial or Construction Debris":"2A863E38-E5D0-455F-87CE-2B75DA29F59A",
    "fragments":"ED401B92-DC24-44C0-A52A-34CE831092BF",
    "Agricultural Waste":"36B2AFEB-7A7C-44B5-A790-5E5C73BA144D",
    "others":"4BEC18FC-BC48-45B7-AFDA-6BA96BD80921",
    "Common Household Items":"C68E90CF-6E65-4474-BC60-72E1C8513F55",
    "plastic":"6961D0DB-928C-419E-9985-98EEEAF552C7",
    "bottles":"9780940B-D06C-4AAB-8003-AB914981E87A",
    "Drinking Bottles":"BCF549A8-AECD-4BC9-B9B8-B94A8F3758D5",
    "Unknown10":"BC7BB564-BE04-4B4B-9913-FF69780B93A6"
    } 
    return switcher.get(label, "nothing")

### Trash INSERT

In [ ]:
''' trashInsert execute INSERT request to dbo.Trash table'''
''' parameter to indsert is a trashTypeId defined in Trash_Type table'''
def trashInsert(trashTypeId,cursor,cnxn):
    cursor.execute("INSERT dbo.Trash (Id,CampaignId,Latitude,Longitude,TrashTypeId,Precision,AI_Version) VALUES (NEWID(),'8D21A132-CF4B-404E-B287-C40A2F12D305','50.797731', '2.179029', ? ,'0.95','0')",trashTypeId)
    cnxn.commit()

In [ ]:
# Executing INSERT for all trash detected in json_prediction from vid1-4K4.MP4
for frame_to_box in json_prediction['detected_trash']:
    print(frame_to_box)
    trashLabel = getTrashLabel(frame_to_box)
    print(trashLabel)
    mapLabel = mapLabel2TrashId(trashLabel)
    print(mapLabel)
    trashInsert(mapLabel,cursor,cnxn)

# Trash End to End

In [ ]:
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobClient
import json 
import os
import pyodbc
import subprocess

In [ ]:
''' blobContainer create a name_list of blobs within container'''
''' params are container name (no full url) & storage conn string'''
def blobInContainer(container,c_string):
    campaign0_container = ContainerClient.from_connection_string(conn_str=c_string, container_name=container)
    blob_list = campaign0_container.list_blobs()
    blob_names_list = []
    for blob in blob_list:
        blob_names_list.append(blob.name)
    return blob_names_list

''' blobInfos provides basic information about a blob object'''
''' params are blob_name only (no full url) & storage conn string'''
def blobInfos(blob_name,c_string):
    blob_video = BlobClient.from_connection_string(conn_str=c_string,container_name="campaign0", blob_name=blob_name)
    blob_video_url = blob_video.url
    blob_video_prop = blob_video.get_blob_properties()
    blob_video_prop_keys = blob_video_prop.keys()
    print("blob name:",blob_name)
    print("blob URL:",blob_video_url)
    print("blob properties:", blob_video_prop)
    print("blob properties keys:", blob_video_prop_keys)


''' downloadBlob from Azure to local file system'''
''' parameter is a blob client object from azure storage sdk'''
def downloadBlob(blobclient):
    with open("/tmp/"+blobclient.blob_name, "wb") as my_blob_dl:
        blob_data = blobclient.download_blob()
        blob_data.readinto(my_blob_dl)
    print("Blob %s downloaded" %blobclient.blob_name)
    print("Blob path: /tmp/%s" %blobclient.blob_name)


''' getPrediction function sends curl request to Surfrider AI'''
''' video name is the name of a locally downloaded video from Azure'''
''' video name is passed as an argument to curl_request script '''
''' curl_request script sends actual POST request to Surfrider AI'''
def getPrediction(video_name):
    curl_request_script = ['./curl_request_param.sh',video_name]
    output = []
    request_answer = subprocess.Popen(curl_request_script, stdout=subprocess.PIPE)
    i = 0
    for line in request_answer.stdout:
        print(line)
        output.append(line)
    return output


''' jsonPrediction cast a prediction from getPrediction function '''
''' from a list prediction to a string then dictionnary with json_loads '''
def jsonPrediction(pred):
    string_prediction = str(pred[0])[2:-3] #removing 2 x first and 3 last characters of pred
    json_prediction = json.loads(string_prediction)
    return json_prediction


''' getTrashLabel return label from a frame_to_box'''
def getTrashLabel(frame_2_box):
    return frame_2_box['label']

''' mapLabelTrashId is a switch that converts label to TrashId'''
''' param is label that comes from AI predictions dictionnary jsonPrediction'''
def mapLabel2TrashId(label):
    switcher = { 
    "Fishing or Hunting":"89B44BAA-69AA-4109-891A-128E012E7E07",
    "Food Packaging":"185FEFA2-EEF2-47A8-873E-26032A4BB3C3",
    "Unknown":"BB4DEA69-218A-40CC-A000-2AE17C37152C",
    "Industrial or Construction Debris":"2A863E38-E5D0-455F-87CE-2B75DA29F59A",
    "fragments":"ED401B92-DC24-44C0-A52A-34CE831092BF",
    "Agricultural Waste":"36B2AFEB-7A7C-44B5-A790-5E5C73BA144D",
    "others":"4BEC18FC-BC48-45B7-AFDA-6BA96BD80921",
    "Common Household Items":"C68E90CF-6E65-4474-BC60-72E1C8513F55",
    "plastic":"6961D0DB-928C-419E-9985-98EEEAF552C7",
    "bottles":"9780940B-D06C-4AAB-8003-AB914981E87A",
    "Drinking Bottles":"BCF549A8-AECD-4BC9-B9B8-B94A8F3758D5",
    "Unknown10":"BC7BB564-BE04-4B4B-9913-FF69780B93A6"
    } 
    return switcher.get(label, "nothing")


''' trashInsert execute INSERT request to dbo.Trash table'''
''' parameter to indsert is a trashTypeId defined in Trash_Type table'''
def trashInsert(cursor,cnxn,trashTypeId):
    cursor.execute("INSERT dbo.Trash (Id,CampaignId,Latitude,Longitude,TrashTypeId,Precision,AI_Version) VALUES (NEWID(),'8D21A132-CF4B-404E-B287-C40A2F12D305','50.797731', '2.179029', ? ,'0.95','0')",trashTypeId)
    cnxn.commit()

In [ ]:
def main():
    
    # blob storage connection string
    connection_string = os.getenv("CONN_STRING")
 
    # get list of blobs in container campaign0
    blobs_campaign0 = blobInContainer('campaign0',connection_string)
    print(blobs_campaign0)

    # get infos of blob video vid1-4K.MP4,vid1-4K2.MP4,vid1-4K3.mp4,vid1-4K4.mp4
    blob_video_name = 'vid1-4K2.mp4'
    blobInfos(blob_video_name,connection_string)

    # download blob video
    blob_video0 = BlobClient.from_connection_string(conn_str=connection_string,container_name="campaign0", blob_name=blob_video_name)
    downloadBlob(blob_video0)

    # get predictions from AI on vid1-4K4.MP4
    prediction = getPrediction(blob_video_name)

    # cast prediction to JSON/Dictionnary format
    json_prediction = jsonPrediction(prediction)
    json_prediction

    # SQL connection
    server = os.getenv("SERVER")
    database = os.getenv("DATABASE")
    username = os.getenv("USERNAME")
    password = os.getenv("PWD")
    driver= '{ODBC Driver 17 for SQL Server}'
    cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()

    # Executing INSERT for all trash detected in json_prediction from vid1-4K4.MP4
    for frame_to_box in json_prediction['detected_trash']:
        print(frame_to_box)
        trashLabel = getTrashLabel(frame_to_box)
        print(trashLabel)
        mapLabel = mapLabel2TrashId(trashLabel)
        print(mapLabel)
        trashInsert(cursor,cnxn,mapLabel)
    
    # Visualize main result within dbo.Trash Table
    cursor.execute("SELECT * FROM dbo.Trash")
    row = cursor.fetchone()
    while row:
        print (str(row[0]))
        row = cursor.fetchone()

In [ ]:
# Execute main function
if __name__ == '__main__':
    main()

## SQL Ops - Optional check

In [ ]:
# SQL Prequesite
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
username = os.getenv("USERNAME")
password = os.getenv("PWD")
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
# SELECT all record from dbo.Trash
cursor.execute("SELECT * FROM dbo.Trash")
row = cursor.fetchone()
while row:
    for i in range(0,5):
        print (str(row[i]))
    row = cursor.fetchone()

In [ ]:
# DELETE ALL RECORD from Trash table
cursor.execute("DELETE FROM dbo.Trash")
cnxn.commit()